Reddit API Wiki: https://github.com/reddit-archive/reddit/wiki

PRAW Github: https://github.com/praw-dev/praw?tab=readme-ov-file

QUICK START: https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

TODO: Pushshift.io API: https://github.com/pushshift/api 提供历史Reddit数据，大规模分析可以试试

In [1]:
!set HTTP_PROXY=http://127.0.0.1: 7890
!set HTTPS_PROXY=https://127.0.0.1: 7890

In [ ]:
# !pip install langchain
# !pip install langchain-openai
# !pip install langchain-deepseek
# !pip install retrying

In [2]:
import praw
import re

## 建立连接

In [3]:
reddit = praw.Reddit(
    client_id="oJd0bJxDJiyBpsGOi7u0rg",
    client_secret="sPJ3VJqMavUNUfXbB7b0ipU34Z-H1g",
    user_agent="testscript by u/fakebot3",
)

## 获取热门帖子 & 评论

In [ ]:
def get_hot_posts(subreddit_name="all", limit=10):
    """
    获取指定子版块的热门帖子\n
    :param subreddit_name: 子版块名称（默认 'all' 即全站）\n
    :param limit: 获取的帖子数量\n
    :return: 帖子信息列表
    """
    subreddit = reddit.subreddit(subreddit_name)
    hot_posts = []
    notFoundList = []

    print(f"🔥 正在获取 r/{subreddit_name} 的热门帖子...\n")

    try:
        for post in subreddit.hot(limit=limit):
        # 提取关键信息
            post_info = {
                "标题": post.title,
                "作者": f"u/{post.author.name}" if post.author else "[已删除]",
                "分数": post.score,
                "评论数": post.num_comments,
                "链接": f"https://reddit.com{post.permalink}",
                "NSFW": post.over_18,
                "发布时间": post.created_utc,  # Unix 时间戳
                "媒体类型": "视频" if post.is_video else "图片" if post.url.endswith(('jpg', 'png', 'gif')) else "文本/链接"
            }
            hot_posts.append(post_info)
    except Exception as e:
        notFoundList.append(subreddit_name)
        return [], notFoundList

    return hot_posts, []

In [40]:
# 示例：获取 r/all 的 10 条热门帖子
posts, notFoundList = get_hot_posts(subreddit_name="all", limit=50)

# 打印结果
for idx, post in enumerate(posts, 1):
    print(f"【{idx}】{post['标题']}")
    print(f"    👍 分数: {post['分数']} | 💬 评论: {post['评论数']} | 👤 作者: {post['作者']}")
    print(f"    🔗 链接: {post['链接']}")
    print(f"    🏷️ NSFW: {'是' if post['NSFW'] else '否'} | 🎬 媒体类型: {post['媒体类型']}\n")

🔥 正在获取 r/all 的热门帖子...

【1】THE DALLAS MAVERICKS HAVE BEEN ELIMINATED FROM THE 2025 NBA CHAMPIONSHIP CONTENTION
    👍 分数: 11275 | 💬 评论: 1202 | 👤 作者: u/Nyhrox
    🔗 链接: https://reddit.com/r/nba/comments/1k2nr7a/the_dallas_mavericks_have_been_eliminated_from/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【2】Wholesome ex-couple
    👍 分数: 33124 | 💬 评论: 255 | 👤 作者: u/Brosse_Adam
    🔗 链接: https://reddit.com/r/MadeMeSmile/comments/1k2m06j/wholesome_excouple/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【3】Japan Just Gave Ukraine $3 Billion from Russian assets
    👍 分数: 7836 | 💬 评论: 104 | 👤 作者: u/Competitive_Mix8130
    🔗 链接: https://reddit.com/r/worldnews/comments/1k2olxh/japan_just_gave_ukraine_3_billion_from_russian/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【4】So….I found this in my garage this morning. Ummm
    👍 分数: 31257 | 💬 评论: 779 | 👤 作者: u/IllDoItTomorr0w
    🔗 链接: https://reddit.com/r/cats/comments/1k2l5mh/soi_found_this_in_my_garage_this_morning_ummm/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【5】NASAs first six women astronauts.

**分数 = 点赞数（Upvotes） - 点踩数（Downvotes）**  

| 指标               | 含义                  | 与分数的关联性         |
|--------------------|-----------------------|-----------------------|
| **评论数**         | 帖子下的评论总数       | 高分数通常伴随高评论数 |
| **Upvote Ratio**   | 点赞/总投票比例        | 比例高 → 争议性低     |
| **NSFW 标记**      | 成人内容标志           | NSFW 帖子的分数可能被抑制 |

In [5]:
def extract_post_id(post_url):
    """
    从 Reddit 帖子链接中提取帖子 ID\n
    :param post_url: Reddit 帖子链接\n
    :return: 帖子 ID
    """
    match = re.search(r'comments/([a-zA-Z0-9]+)/', post_url)
    return match.group(1) if match else None

In [6]:
def get_comments(post_id):
    """
    获取指定帖子的用户评论\n
    :param post_id: 帖子 ID\n
    :return: 评论信息列表
    """
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=None)  # 展开所有评论
    comments = []

    for comment in submission.comments.list():
        comment_info = {
            "作者": f"u/{comment.author.name}" if comment.author else "[已删除]",
            "内容": comment.body,
            "分数": comment.score,
            "发布时间": comment.created_utc,  # Unix 时间戳
        }
        comments.append(comment_info)

    return comments

In [7]:
# 通过 id 获得帖子的内容
def get_content(post_id):
    """
    获取指定帖子的内容\n
    :param post_id: 帖子 ID\n
    :return: 帖子内容
    """
    submission = reddit.submission(id=post_id)
    content = submission.selftext
    return content

In [41]:
def get_posts_title_content(posts):
    """
    获取帖子内容\n
    :param posts: 帖子信息列表
    :return: 帖子 ID、标题和内容的元组列表
    """
    posts_id_title_contents = []
    for post in posts:
        submission = reddit.submission(extract_post_id(post['链接']))
        content = submission.selftext
        posts_id_title_contents.append((extract_post_id(post['链接']), post['标题'], content, post['链接']))
    return posts_id_title_contents

In [ ]:
comments = get_comments(extract_post_id(posts[9]['链接']))
# 打印评论结果
for idx, comment in enumerate(comments, 1):
    print(f"【{idx}】{comment['内容']}")
    print(f"    👍 分数: {comment['分数']} | 👤 作者: {comment['作者']}")
    print(f"    ⏰ 发布时间: {comment['发布时间']}\n")

In [ ]:
# TODO 国内wb，热搜
# https://open.weibo.com/wiki/SDK
# 只有 移动端的SDK 和 JS，遂放弃

# https://github.com/dataabc/weibo-crawler?tab=readme-ov-file#api%E6%9C%8D%E5%8A%A1
# 这种项目只有爬取某个指定用户的发帖，如果要做实际的 user persona 可以考虑

In [9]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
os.environ["DEEPSEEK_API_KEY"] = "sk-49fab4d5c2aa476f8233bd0863fa5984"
os.environ["DEEPSEEK_BASE_URL"] = "https://api.deepseek.com/v1"

deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=1)

In [ ]:
prompt = f"""我需要你根据用户输入的喜好领域，生成尽可能多相关的细分关键词。请遵循以下规则：

            1. 输出格式：纯文本，逗号分隔，不编号
            2. 关键词类型需包含：
            - 核心术语
            - 相关子领域
            - 工具/设备
            - 技术/方法
            - 流派/风格
            - 知名品牌
            - 代表人物
            - 关联概念
            3. 数量要求：至少30个不重复关键词

            参考案例：
            输入：旅游
            输出：[自由行, 背包客, 自驾游, 民宿体验, 景点打卡, 旅行摄影, 当地美食, 徒步路线, 文化遗产, 免税购物, 签证攻略, 旅行保险, 航拍设备, 旅行博主, 穷游指南, 邮轮度假, 户外装备, 语言翻译APP, 时差调整, 旅行纪念品, 地理定位, 生态旅游, 沙发客, 旅行vlog, 旅行穿搭, 货币兑换, 旅行插头转换器, 孤独星球指南, 旅行急救包, 数字游民]

            输入：咖啡
            输出：[手冲咖啡, 拉花艺术, 咖啡烘焙度, 单品豆, 意式浓缩, 冷萃技术, 咖啡因含量, 咖啡渣利用, 摩卡壶, 法压壶, 咖啡师认证, 咖啡豆产区, 第三波咖啡浪潮, 咖啡杯测, 咖啡因过敏, 咖啡馆设计, 挂耳咖啡, 咖啡调糖, 咖啡伴侣, 咖啡机维护, 咖啡品鉴术语, 咖啡因代谢, 咖啡主题旅行, 咖啡烘焙机, 咖啡师大赛, 咖啡冥想, 咖啡因耐受, 咖啡渣去角质, 咖啡风味轮, 咖啡豆保存罐]

            现在请处理新输入：
            输入：摄影
            输出：[]
            """

response = deepseek_chat(prompt)

print(response)

In [10]:
# 调用 DeepSeek API 通过 keyword 进行衍生
def get_keywords_set(keyword):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an AI assistant, please answer user's question."),
            ("user", "{input}"),
        ]
    )
    model = ChatDeepSeek(model="deepseek-chat")

    chain = prompt | model

    response = chain.invoke(
        {
            "input": f"""我需要你根据用户输入的喜好领域，生成尽可能多相关的细分关键词。请遵循以下规则：

            1. 输出格式：纯文本，逗号分隔，不编号
            2. 关键词类型需包含：
            - 核心术语
            - 相关子领域
            - 工具/设备
            - 技术/方法
            - 流派/风格
            - 知名品牌
            - 代表人物
            - 关联概念
            3. 数量要求：至少30个，最多40个不重复关键词

            参考案例：
            输入：旅游
            输出：[自由行, 背包客, 自驾游, 民宿体验, 景点打卡, 旅行摄影, 当地美食, 徒步路线, 文化遗产, 免税购物, 签证攻略, 旅行保险, 航拍设备, 旅行博主, 穷游指南, 邮轮度假, 户外装备, 语言翻译APP, 时差调整, 旅行纪念品, 地理定位, 生态旅游, 沙发客, 旅行vlog, 旅行穿搭, 货币兑换, 旅行插头转换器, 孤独星球指南, 旅行急救包, 数字游民]

            输入：咖啡
            输出：[手冲咖啡, 拉花艺术, 咖啡烘焙度, 单品豆, 意式浓缩, 冷萃技术, 咖啡因含量, 咖啡渣利用, 摩卡壶, 法压壶, 咖啡师认证, 咖啡豆产区, 第三波咖啡浪潮, 咖啡杯测, 咖啡因过敏, 咖啡馆设计, 挂耳咖啡, 咖啡调糖, 咖啡伴侣, 咖啡机维护, 咖啡品鉴术语, 咖啡因代谢, 咖啡主题旅行, 咖啡烘焙机, 咖啡师大赛, 咖啡冥想, 咖啡因耐受, 咖啡渣去角质, 咖啡风味轮, 咖啡豆保存罐]

            现在请处理新输入：
            输入：{keyword}
            输出：[]
            """
        }
    )

    # 对 response里输出的 "[]" 中的内容进行正则提取，转化为list
    response = response.content.replace("[]", "").replace("，", ",").replace(" ", "").replace("\n", "").replace("[", "").replace("]", "").replace("'", "").replace('"', "")
    response = re.findall(r"[^,]+", response)
    return response

# 另外一个不调用 langchain 的版本
def get_keywords_set_without_langchain(keyword="摄影"):
    prompt = f"""我需要你根据用户输入的喜好领域，生成尽可能多相关的细分关键词。请遵循以下规则：

            1. 输出格式：纯文本，逗号分隔，不编号
            2. 关键词类型需包含：
            - 核心术语
            - 相关子领域
            - 工具/设备
            - 技术/方法
            - 流派/风格
            - 知名品牌
            - 代表人物
            - 关联概念
            3. 数量要求：至少30个，最多40个不重复关键词

            参考案例：
            输入：旅游
            输出：[自由行, 背包客, 自驾游, 民宿体验, 景点打卡, 旅行摄影, 当地美食, 徒步路线, 文化遗产, 免税购物, 签证攻略, 旅行保险, 航拍设备, 旅行博主, 穷游指南, 邮轮度假, 户外装备, 语言翻译APP, 时差调整, 旅行纪念品, 地理定位, 生态旅游, 沙发客, 旅行vlog, 旅行穿搭, 货币兑换, 旅行插头转换器, 孤独星球指南, 旅行急救包, 数字游民]

            输入：咖啡
            输出：[手冲咖啡, 拉花艺术, 咖啡烘焙度, 单品豆, 意式浓缩, 冷萃技术, 咖啡因含量, 咖啡渣利用, 摩卡壶, 法压壶, 咖啡师认证, 咖啡豆产区, 第三波咖啡浪潮, 咖啡杯测, 咖啡因过敏, 咖啡馆设计, 挂耳咖啡, 咖啡调糖, 咖啡伴侣, 咖啡机维护, 咖啡品鉴术语, 咖啡因代谢, 咖啡主题旅行, 咖啡烘焙机, 咖啡师大赛, 咖啡冥想, 咖啡因耐受, 咖啡渣去角质, 咖啡风味轮, 咖啡豆保存罐]

            现在请处理新输入：
            输入：{keyword}
            输出：[]
            """

    response = deepseek_chat(prompt, temperature=1)
    # 对 response里输出的 "[]" 中的内容进行正则提取，转化为list，根据逗号分隔
    response = response.content.replace("[]", "").replace("，", ",").replace(" ", "").replace("\n", "").replace("[", "").replace("]", "").replace("'", "").replace('"', "")
    response = re.findall(r"[^,]+", response)
    return response


In [11]:
def translate(keywordsList):
    """
    将关键词列表翻译为中文或英文\n
    :param keywordsList: 关键词列表
    :return: 翻译后的关键词列表
    """
    # 使用 deepseek_chat 进行翻译
    wordStr = "[" + ", ".join(keywordsList) + "]"
    prompt = f"""我需要你根据参考格式将以下中文关键词翻译为英文，或将英文关键词翻译为中文: {wordStr}
        参考格式1（中译英）：
        输入：[自由行, 背包客, 自驾游, 民宿体验, 景点打卡, 旅行摄影, 当地美食, 徒步路线, 文化遗产, 免税购物, 签证攻略, 旅行保险, 航拍设备, 旅行博主, 穷游指南, 邮轮度假, 户外装备, 语言翻译APP, 时差调整, 旅行纪念品, 地理定位, 生态旅游, 沙发客, 旅行vlog, 旅行穿搭, 货币兑换, 旅行插头转换器, 孤独星球指南, 旅行急救包, 数字游民]
        输出：[Free travel, Backpacker, Self-driving tour, Homestay experience, Scenic spot check-in, Travel photography, Local food, Hiking route, Cultural heritage, Duty-free shopping, Visa guide, Travel insurance, Aerial photography equipment, Travel blogger, Budget travel guide, Cruise vacation, Outdoor equipment, Language translation APP, Jet lag adjustment, Travel souvenirs, Geolocation, Ecotourism, Couchsurfing, Travel vlog, Travel outfit, Currency exchange, Travel plug adapter, Lonely Planet guidebook, Travel first aid kit, Digital nomad]
    
        参考格式2（英译中）：
        输入：[Digital camera, SLR camera, micro single camera, lens selection, aperture control, shutter speed, ISO sensitivity, composition skills, Light use, portrait photography, landscape photography, documentary photography, street photography, night shooting, long exposure, HDR technology, panoramic shooting, drone aerial photography, tripod use, Filter Effects, Post Repainting, Lightroom tutorial, Photoshop tips, Black and White Photography, film camera, Darkroom techniques, Studio Lighting, Reflector Use, Photography Backpack, Memory Card Selection, Battery life, Photography competitions, Photography Exhibitions, photography Books, Photography magazines, Photographer Community, Photography Workshop, Photography Travel, Underwater photography, Macro photography, Time-lapse photography, photography copyright, Photo printing, Photo frame selection, Photography history, famous photographers, photography genres, Photography Ethics, photography equipment rental, Photography APP recommendation]
        输出：[数码相机, 单反相机, 微单相机, 镜头选择, 光圈控制, 快门速度, ISO感光度, 构图技巧, 光线运用, 人像摄影, 风景摄影, 纪实摄影, 街头摄影, 夜景拍摄, 长曝光, HDR技术, 全景拍摄, 无人机航拍, 三脚架使用, 滤镜效果, 后期修图, Lightroom教程, Photoshop技巧, 黑白摄影, 胶片相机, 暗房技术, 摄影棚灯光, 反光板运用, 摄影背包, 存储卡选择, 电池续航, 摄影比赛, 摄影展览, 摄影书籍, 摄影杂志, 摄影师社区, 摄影工作坊, 摄影旅行, 水下摄影, 微距摄影, 延时摄影, 摄影版权, 照片打印, 相框选择, 摄影历史, 著名摄影师, 摄影流派, 摄影伦理, 摄影器材租赁, 摄影APP推荐]
        
        注意：只需要输出[]中的内容，其他的内容都不需要。
    """
    print(prompt)
    response = deepseek_chat(prompt)
    response = response.content.replace("[]", "").replace("，", ",").replace("\n", "").replace("[", "").replace("]", "").replace("'", "").replace('"', "")

    if response:
        response = response.split(", ")
    else:
        response = []

    return response


# response = translate(
#     [
#         "手冲咖啡",
#         "拉花艺术",
#         "咖啡烘焙度",
#         "单品豆",
#         "意式浓缩",
#         "冷萃技术",
#         "咖啡因含量",
#     ]
# )
# print(response)

In [12]:
# 从数据集中获取一个用户的喜好领域关键词
keyword = "摄影"

# 调用大模型衍生出可能的类似关键词集合
keywordsList = get_keywords_set_without_langchain(keyword)
keywordsList = translate(keywordsList) + keywordsList
# 在爬取的热点话题中 re 匹配
for keyword in keywordsList:
    print(keyword, end=" ")

C:\Users\towyz\AppData\Local\Temp\ipykernel_40340\4242613338.py:76: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = deepseek_chat(prompt, temperature=1)


我需要你根据参考格式将以下中文关键词翻译为英文，或将英文关键词翻译为中文: [单反相机, 微单相机, 镜头选择, 光圈优先, 快门速度, ISO感光度, 构图法则, 黄金分割, 人像摄影, 风光摄影, 纪实摄影, 街头摄影, 黑白摄影, 长曝光, HDR技术, 无人机航拍, 三脚架, 滤镜系统, 闪光灯技巧, 反光板, 摄影包, 照片后期, Lightroom修图, Photoshop技巧, 色彩校正, 胶片摄影, 暗房工艺, 摄影工作室, 摄影比赛, 摄影集出版, 摄影展览, 摄影杂志, 摄影旅行, 摄影教学, 摄影博客, 摄影社区, 摄影版权, 摄影合同, 摄影器材租赁, 摄影保险]
        参考格式1（中译英）：
        输入：[自由行, 背包客, 自驾游, 民宿体验, 景点打卡, 旅行摄影, 当地美食, 徒步路线, 文化遗产, 免税购物, 签证攻略, 旅行保险, 航拍设备, 旅行博主, 穷游指南, 邮轮度假, 户外装备, 语言翻译APP, 时差调整, 旅行纪念品, 地理定位, 生态旅游, 沙发客, 旅行vlog, 旅行穿搭, 货币兑换, 旅行插头转换器, 孤独星球指南, 旅行急救包, 数字游民]
        输出：[Free travel, Backpacker, Self-driving tour, Homestay experience, Scenic spot check-in, Travel photography, Local food, Hiking route, Cultural heritage, Duty-free shopping, Visa guide, Travel insurance, Aerial photography equipment, Travel blogger, Budget travel guide, Cruise vacation, Outdoor equipment, Language translation APP, Jet lag adjustment, Travel souvenirs, Geolocation, Ecotourism, Couchsurfing, Travel vlog, Travel outfit, Currency exchange, Travel plug ada

In [13]:
def get_posts_by_keywords(keywordsList):
    """
    根据关键词列表获取相关帖子\n
    :param keywordsList: 关键词列表
    :return: 帖子信息列表
    """
    posts = []
    for keyword in keywordsList:
        hot_posts, notFoundList = get_hot_posts(subreddit_name=keyword, limit=10)
        if notFoundList:
            print(f"🥲 未找到子版块：{notFoundList}\n")
        posts.extend(hot_posts)
    return posts


In [ ]:
posts = get_posts_by_keywords(keywordsList)

🔥 正在获取 r/DSLR camera 的热门帖子...

🥲 未找到子版块：['DSLR camera']

🔥 正在获取 r/Mirrorless camera 的热门帖子...

🥲 未找到子版块：['Mirrorless camera']

🔥 正在获取 r/Lens selection 的热门帖子...

🥲 未找到子版块：['Lens selection']

🔥 正在获取 r/Aperture priority 的热门帖子...

🥲 未找到子版块：['Aperture priority']

🔥 正在获取 r/Shutter speed 的热门帖子...

🥲 未找到子版块：['Shutter speed']

🔥 正在获取 r/ISO sensitivity 的热门帖子...

🥲 未找到子版块：['ISO sensitivity']

🔥 正在获取 r/Composition rules 的热门帖子...

🥲 未找到子版块：['Composition rules']

🔥 正在获取 r/Golden ratio 的热门帖子...

🥲 未找到子版块：['Golden ratio']

🔥 正在获取 r/Portrait photography 的热门帖子...

🥲 未找到子版块：['Portrait photography']

🔥 正在获取 r/Landscape photography 的热门帖子...

🥲 未找到子版块：['Landscape photography']

🔥 正在获取 r/Documentary photography 的热门帖子...

🥲 未找到子版块：['Documentary photography']

🔥 正在获取 r/Street photography 的热门帖子...

🥲 未找到子版块：['Street photography']

🔥 正在获取 r/Black and white photography 的热门帖子...

🥲 未找到子版块：['Black and white photography']

🔥 正在获取 r/Long exposure 的热门帖子...

🥲 未找到子版块：['Long exposure']

🔥 正在获取 r/HDR technology 的热门帖子...



In [26]:
# for post in posts:
#     print(f"帖子标题: {post['标题']}, 帖子链接: {post['链接']}")

print(posts[19])
len(posts)


{'标题': "Jessica: ''Abrego Garcia's legal status was approved by the Trump administration in 2019, and every year since then, he has checked in with the departed of homeland security, and no one has said anything about him being a gang banger.''", '作者': 'u/RoyalChris', '分数': 22110, '评论数': 595, '链接': 'https://reddit.com/r/WomenInNews/comments/1k2hta6/jessica_abrego_garcias_legal_status_was_approved/', 'NSFW': False, '发布时间': 1745016292.0, '媒体类型': '视频'}


50

In [42]:
posts = posts[0: 50]
posts_id_title_content = get_posts_title_content(posts)


In [46]:
posts_id_title_content_not_empty = [post for post in posts_id_title_content if post[2] != ""]
print(posts_id_title_content_not_empty[1])

('1k2l5mh', 'So….I found this in my garage this morning. Ummm', 'We have 3 dogs but no cats. We have never even had a cat before. Mom hisses a lot at us but we put kitty food and water down and they calmed down a bit. ', 'https://reddit.com/r/cats/comments/1k2l5mh/soi_found_this_in_my_garage_this_morning_ummm/')
